# T1SS Expression Analysis of Flow Cytometry Data

This is a data analysis pipeline I developed to analyse flow cytometry data for the Type I Secretion System Expression in E. coli.

### Import Modules

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### Import Data

In [ ]:
df = pd.read_csv("Flow_Cytometry_Expression_Data.csv")

In [ ]:
# Get non-control data
experiment_df = df.loc[df["Control?"] == "No"]

### Apply Standardisation/Normalisation

In [ ]:
# Normalise T1SS data
def normalise_T1SS(value,df=experiment_df):
    """
    Function takes T1SS values and standardises based on excess mean and standard deviation
    """
    excess_mean = df["Excess(%)"].mean()
    excess_std = df["Excess(%)"].std()
    excess_n = len(df["Excess(%)"])
    normal_value = (value - excess_mean)/(excess_std/np.sqrt(excess_n))
    
    return normal_value

In [ ]:
# Make normalised column
experiment_df["Normalised_T1SS(%)"] = experiment_df["T1SS(%)"].apply(normalise_T1SS)

In [ ]:
# Plot normalised values
experiment_df["Normalised_T1SS(%)"].hist()

In [ ]:
# Plot unnormalised values
experiment_df["T1SS(%)"].hist()

In [ ]:
# Plot excess (Q4) values
experiment_df["Excess(%)"].hist()

In [ ]:
# Calculate difference between unnormalised T1SS and Excess (Q4)
experiment_df["T1SS-Excess(%)"] = experiment_df["T1SS(%)"] - experiment_df["Excess(%)"]

In [ ]:
# Plot difference between unnormalised T1SS and Excess (Q4)
experiment_df["T1SS-Excess(%)"].hist()

In [ ]:
# Extract positive values in difference between T1SS and Excess (Q4)
new_experiment_df = experiment_df[experiment_df["T1SS-Excess(%)"]>=0]

In [ ]:
# Plot positive values in difference between T1SS and Excess (Q4)
new_experiment_df["T1SS-Excess(%)"].hist()

Plots shows standardisation/normalisation is correct.

### Exploratory Data Analysis

In [ ]:
# Extract positive T1SS expression values
remove_df = experiment_df.loc[experiment_df["Normalised_T1SS(%)"] >= 0]

In [ ]:
# Extract BL21 Strain results
bl21_remove_df = remove_df[remove_df["Strain"] == "BL21(DE3)"]

In [ ]:
# Extract MG1655 results - parent and cardiolipin deficient
mask = (remove_df["Strain"] == "MG1655 Parent") | (remove_df["Strain"] == "MG1655 Delta cls")
mg1655_remove_df = remove_df[mask]

In [ ]:
# Extract MG1655 parent only
mg1655_parent = remove_df[remove_df["Strain"] == "MG1655 Parent"]

In [ ]:
# Extract MG1655 cardiolipin deficient only
mg1655_deltacls = remove_df[remove_df["Strain"] == "MG1655 Delta cls"]

In [ ]:
# save new df
remove_df.to_csv("Normalised Experimental Data.csv", index=False)

In [ ]:
# drop columns for plotting
remove_df.drop(["Chloramphenicol_Concentration(ug/ml)"],axis=1,inplace=True)
remove_df.drop(['Date(DDMMYYYY)','Total(%)','Control?','Primary Ig?','Secondary Ig?','Repeat_Measurement?'],axis=1,inplace=True)

### Poisson Distribution Fits

In [ ]:
# Round the counts to fit poisson distribution
remove_df["Rounded_Normalised_T1SS(%)"] = remove_df["Normalised_T1SS(%)"].round(0)
bl21_remove_df["Rounded_Normalised_T1SS(%)"] = bl21_remove_df["Normalised_T1SS(%)"].round(0)
mg1655_remove_df["Rounded_Normalised_T1SS(%)"] = mg1655_remove_df["Normalised_T1SS(%)"].round(0)

In [ ]:
# Extract values and counts for plotting
remove_df_plot_data = remove_df["Rounded_Normalised_T1SS(%)"].value_counts().to_dict()
bl21_plot_data = bl21_remove_df["Rounded_Normalised_T1SS(%)"].value_counts().to_dict()
mg1655_plot_data = mg1655_remove_df["Rounded_Normalised_T1SS(%)"].value_counts().to_dict()

In [ ]:
# Find probs by dividing freq by total freq - all strains
remove_df_scaled_plot_data = dict()
for key, val in remove_df_plot_data.items():
    new_val = val/sum(remove_df_plot_data.values())
    remove_df_scaled_plot_data[key] = new_val

In [ ]:
# Find probs by dividing freq by total freq - bl21
bl21_scaled_plot_data = dict()
for key, val in bl21_plot_data.items():
    new_val = val/sum(bl21_plot_data.values())
    bl21_scaled_plot_data[key] = new_val

In [ ]:
# Find probs by dividing freq by total freq - mg1655
mg1655_scaled_plot_data = dict()
for key, val in mg1655_plot_data.items():
    new_val = val/sum(mg1655_plot_data.values())
    mg1655_scaled_plot_data[key] = new_val

In [ ]:
# Three lamda poisson distribution
def poisson(x,lam1,lam2,lam3):
    y = (((lam1**x) * np.exp(-lam1))/np.math.factorial(x)) + (((lam2**x) * np.exp(-lam2))/np.math.factorial(x)) + (((lam3**x) * np.exp(-lam3))/np.math.factorial(x))
    return y

In [ ]:
# Calculate poisson probs - all strains
all_strain_x = list(remove_df_scaled_plot_data.keys())
all_strain_y = list(remove_df_scaled_plot_data.values())
all_strain_y_prob = []
# Find probs for x
for elem in all_strain_x:
    all_strain_prob_x = poisson(elem,5,20,48)
    all_strain_y_prob.append(all_strain_prob_x)

In [ ]:
# Calculate poisson probs - BL21 Strain
bl21_x = list(bl21_scaled_plot_data.keys())
bl21_y = list(bl21_scaled_plot_data.values())
bl21_y_prob = []
# Find probs for x
for elem in bl21_x:
    bl21_prob_x = poisson(elem,8,25,48)
    bl21_y_prob.append(bl21_prob_x)

In [ ]:
# Calculate poisson probs - MG1655 Strain
mg1655_x = list(mg1655_scaled_plot_data.keys())
mg1655_y = list(mg1655_scaled_plot_data.values())
mg1655_y_prob = []
# Find probs for x
for elem in mg1655_x:
    mg1655_prob_x = poisson(elem,5,15,35)
    mg1655_y_prob.append(mg1655_prob_x)

In [ ]:
# Rescale poisson probs - All Strains
rescale_all_strain_y_prob = []
for elem in all_strain_y_prob:
    new_elem = elem/sum(all_strain_y_prob)
    rescale_all_strain_y_prob.append(new_elem)

In [ ]:
# Rescale poisson probs - BL21
rescale_bl21_y_prob = []
for elem in bl21_y_prob:
    new_elem = elem/sum(bl21_y_prob)
    rescale_bl21_y_prob.append(new_elem)

In [ ]:
# Rescale poisson probs - MG1655
rescale_mg1655_y_prob = []
for elem in mg1655_y_prob:
    new_elem = elem/sum(mg1655_y_prob)
    rescale_mg1655_y_prob.append(new_elem)

In [ ]:
# Plotting - All Strain
plt.bar(all_strain_x,all_strain_y,label='data')
plt.bar(all_strain_x,rescale_all_strain_y_prob,alpha=0.25,color='red',label='fit')
plt.legend()

In [ ]:
# Plotting - BL21
plt.bar(bl21_x,bl21_y,label='data')
plt.bar(bl21_x,rescale_bl21_y_prob,alpha=0.25,color='red',label='fit')
plt.legend()

In [ ]:
# Plotting - MG1655
plt.bar(mg1655_x,mg1655_y,label='data')
plt.bar(mg1655_x,rescale_mg1655_y_prob,alpha=0.25,color='red',label='fit')
plt.legend()

In [ ]:
# Prepare dataframes for chi square test
d_all_strain = {'Normalised_T1SS(%)':all_strain_x, 'Obs_Freq':list(remove_df_plot_data.values()),'Prob':all_strain_y, 'Fit':rescale_all_strain_y_prob}
d_bl21 = {'Normalised_T1SS(%)':bl21_x, 'Obs_Freq':list(bl21_plot_data.values()), 'Prob':bl21_y, 'Fit':rescale_bl21_y_prob}
d_mg1655 = {'Normalised_T1SS(%)':mg1655_x, 'Obs_Freq':list(mg1655_plot_data.values()),'Prob':mg1655_y, 'Fit':rescale_mg1655_y_prob}

In [ ]:
# Dataframes to test chi square goodness of fit
data_all_strain = pd.DataFrame(data=d_all_strain)
data_bl21 = pd.DataFrame(data=d_bl21)
data_mg1655 = pd.DataFrame(data=d_mg1655)

In [ ]:
# Calculate expected frequencies
data_all_strain["Expct_Freq"] = (data_all_strain["Fit"] * sum(data_all_strain["Obs_Freq"])).round(1)
data_bl21["Expct_Freq"] = (data_bl21["Fit"] * sum(data_bl21["Obs_Freq"])).round(1)
data_mg1655["Expct_Freq"] = (data_mg1655["Fit"] * sum(data_mg1655["Obs_Freq"])).round(1)

In [ ]:
# Calculate square of observed minus expected
data_all_strain["Obs-Expected_Squared"] = (data_all_strain["Obs_Freq"] - data_all_strain["Expct_Freq"])**2
data_bl21["Obs-Expected_Squared"] = (data_bl21["Obs_Freq"] - data_bl21["Expct_Freq"])**2
data_mg1655["Obs-Expected_Squared"] = (data_mg1655["Obs_Freq"] - data_mg1655["Expct_Freq"])**2

In [ ]:
# Calculate square obs mins expct divided by expct
data_all_strain["Test"] = data_all_strain["Obs-Expected_Squared"]/data_all_strain["Expct_Freq"]
data_bl21["Test"] = data_bl21["Obs-Expected_Squared"]/data_bl21["Expct_Freq"]
data_mg1655["Test"] = data_mg1655["Obs-Expected_Squared"]/data_mg1655["Expct_Freq"]

In [ ]:
sum(data_all_strain["Test"])

In [ ]:
sum(data_bl21["Test"])

In [ ]:
sum(data_mg1655["Test"])

Don't reject null and thus parameterisation is fine.

### Exploratory Plots

In [ ]:
# Plot CaCl2 for BL21 strain
fig = sns.catplot(x="Cacl(mM)", y="Normalised_T1SS(%)", kind='box', data=bl21_remove_df)
#fig.savefig("BL21 CaCl Plots.png", dpi=300)

In [ ]:
# Plot T1SS by E. coli strain
fig = sns.catplot(x="Normalised_T1SS(%)", y="Strain", kind='box', height=4, aspect=2, data=remove_df)
#fig.savefig("All Strain T1SS Plots.png", dpi=300)

In [ ]:
# Plot T1SS by expression order for BL21 strain
fig = sns.catplot(x="Expression_Order", y="Normalised_T1SS(%)", kind='box', data=bl21_remove_df)
#fig.savefig("BL21 Expression Order.png", dpi=300)

In [ ]:
# Plot by carbenicillin concentration for BL21 strain
fig = sns.catplot(x="Carbenicillin_Concentration(ug/ml)", y="Normalised_T1SS(%)", kind='boxen', data=bl21_remove_df)
#fig.savefig("BL21 Carb Conc.png", dpi=300)

In [ ]:
# Plot by kanamycin concentration for BL21 strain
fig = sns.catplot(x="Kanamycin_Concentration(ug/ml)", y="Normalised_T1SS(%)", kind='boxen', data=bl21_remove_df)
#fig.savefig("BL21 Kan Conc.png", dpi=300)

In [ ]:
# Plot by total expression time for BL21 strain
fig = sns.catplot(x="Total_Expression_Time(hr)", y="Normalised_T1SS(%)", kind='box', data=bl21_remove_df)
#fig.savefig("BL21 Time Course.png", dpi=300)

### Data Grouping for complex plots

In [ ]:
# Group data - BL21 strain
group_anti = bl21_remove_df.groupby(["Carbenicillin_Concentration(ug/ml)", "Kanamycin_Concentration(ug/ml)", "Total_Expression_Time(hr)", "Cacl(mM)"])["Normalised_T1SS(%)"].mean().unstack()
group_anti.reset_index(col_level=["Carbenicillin_Concentration(ug/ml)", "Kanamycin_Concentration(ug/ml)", "Total_Expression_Time(hr)"],inplace=True)

In [ ]:
# Get columns - BL21
group_anti.columns = ['Carbenicillin_Concentration(ug/ml)','Kanamycin_Concentration(ug/ml)','Total_Expression_Time(hr)','0 mM CaCl', '1 mM CaCl', '2 mM CaCl', '5 mM CaCl', '8 mM CaCl', '10 mM CaCl']

In [ ]:
# Combine by antibiotics - Bl21
group_anti["Carbenicillin_+_Kanamycin_Concentration(ug/ml)"] = group_anti["Carbenicillin_Concentration(ug/ml)"].astype(str) + "  ug/ml carbenicillin + " + group_anti["Kanamycin_Concentration(ug/ml)"].astype(str) + " ug/ml kanamycin"

In [ ]:
# Group data - MG1655
mg1655_group_anti = mg1655_remove_df.groupby(["Strain","Carbenicillin_Concentration(ug/ml)", "Kanamycin_Concentration(ug/ml)", "Total_Expression_Time(hr)", "Cacl(mM)"])["Normalised_T1SS(%)"].mean().unstack()
mg1655_group_anti.reset_index(col_level=['Strain', 'Carbenicillin_Concentration(ug/ml)', 'Kanamycin_Concentration(ug/ml)', 'Total_Expression_Time(hr)'],inplace=True)

In [ ]:
# Get columns - MG1655
mg1655_group_anti.columns = ['Strain','Carbenicillin_Concentration(ug/ml)','Kanamycin_Concentration(ug/ml)','Total_Expression_Time(hr)','0 mM CaCl', '2 mM CaCl', '5 mM CaCl', '8 mM CaCl', '10 mM CaCl']

In [ ]:
# Combine by antibiotics - MG1655
mg1655_group_anti["Carbenicillin_+_Kanamycin_Concentration(ug/ml)"] = mg1655_group_anti["Carbenicillin_Concentration(ug/ml)"].astype(str) + "  ug/ml carbenicillin + " + mg1655_group_anti["Kanamycin_Concentration(ug/ml)"].astype(str) + " ug/ml kanamycin"

In [ ]:
# Group data - BL21 and MG1655 strains
all_anti = remove_df.groupby(["Strain","Carbenicillin_Concentration(ug/ml)", "Kanamycin_Concentration(ug/ml)", "Total_Expression_Time(hr)", "Cacl(mM)"])["Normalised_T1SS(%)"].mean().unstack()
all_anti.reset_index(col_level=["Carbenicillin_Concentration(ug/ml)", "Kanamycin_Concentration(ug/ml)", "Total_Expression_Time(hr)"],inplace=True)

In [ ]:
# Get columns - BL21 and MG1655
all_anti.columns = ['Strain','Carbenicillin_Concentration(ug/ml)','Kanamycin_Concentration(ug/ml)','Total_Expression_Time(hr)','0 mM CaCl', '1 mM CaCl', '2 mM CaCl', '5 mM CaCl', '8 mM CaCl', '10 mM CaCl']

In [ ]:
# Separate MG1655 by parent and cardiolipin deficient
mg1655_parent_group_anti = mg1655_group_anti[mg1655_group_anti["Strain"] == "MG1655 Parent"]
mg1655_cls_group_anti = mg1655_group_anti[mg1655_group_anti["Strain"] == "MG1655 Delta cls"]

### Complex Plots

In [ ]:
# Plot by total expression time - Bl21
fig = sns.pairplot(group_anti,
            x_vars=["Carbenicillin_Concentration(ug/ml)", "Kanamycin_Concentration(ug/ml)"],
            y_vars=["0 mM CaCl", "1 mM CaCl", "2 mM CaCl", "5 mM CaCl", "8 mM CaCl", "10 mM CaCl"],
            hue="Total_Expression_Time(hr)")
#fig.savefig("BL21 CaCl and Antibiotic Conc with Expression Time.png", dpi=300)

In [ ]:
# Plot by antibiotic concentration - Bl21
g = sns.pairplot(group_anti,
            x_vars=["0 mM CaCl", "1 mM CaCl", "2 mM CaCl", "5 mM CaCl", "8 mM CaCl", "10 mM CaCl"],
            y_vars=["Total_Expression_Time(hr)"],
            hue="Carbenicillin_+_Kanamycin_Concentration(ug/ml)")
g.savefig("BL21 CaCl and Expression Time coloured by antibiotic conc.png", dpi=300)

In [ ]:
# Plot by total expression time - MG1655
g = sns.pairplot(mg1655_group_anti,
            x_vars=["0 mM CaCl", "2 mM CaCl", "5 mM CaCl", "8 mM CaCl"],
            y_vars=["Total_Expression_Time(hr)"],
            hue="Strain")
g.savefig("MG1655 CaCl and Expression Time.png", dpi=300)

In [ ]:
# Plot by CaCl2 - MG1655
fig = sns.pairplot(mg1655_group_anti,
            x_vars=["Carbenicillin_Concentration(ug/ml)", "Kanamycin_Concentration(ug/ml)","Total_Expression_Time(hr)"],
            y_vars=["0 mM CaCl", "2 mM CaCl", "5 mM CaCl", "8 mM CaCl", "10 mM CaCl"],
            hue="Strain")
fig.savefig("MG1655 CaCl and Antibiotic Conc with Expression Time.png", dpi=300)

In [ ]:
# Plot by total expression time - MG1655 parent only
fig = sns.pairplot(mg1655_parent_group_anti,
            x_vars=["Carbenicillin_Concentration(ug/ml)", "Kanamycin_Concentration(ug/ml)"],
            y_vars=["0 mM CaCl", "2 mM CaCl", "5 mM CaCl", "8 mM CaCl", "10 mM CaCl"],
            hue="Total_Expression_Time(hr)")
fig.savefig("MG1655 Parent CaCl and Antibiotic Conc with Expression Time.png", dpi=300)

In [ ]:
# Plot by total expression time - MG1655 cardiolipin deficient only
fig = sns.pairplot(mg1655_cls_group_anti,
            x_vars=["Carbenicillin_Concentration(ug/ml)", "Kanamycin_Concentration(ug/ml)"],
            y_vars=["0 mM CaCl", "2 mM CaCl", "5 mM CaCl", "8 mM CaCl", "10 mM CaCl"],
            hue="Total_Expression_Time(hr)")
fig.savefig("MG1655 Delta cls CaCl and Antibiotic Conc with Expression Time.png", dpi=300)

In [ ]:
# Plot by CaCl2 - all strains
fig = sns.pairplot(all_anti,
            x_vars=["Carbenicillin_Concentration(ug/ml)", "Kanamycin_Concentration(ug/ml)", "Total_Expression_Time(hr)"],
            y_vars=["0 mM CaCl", "1 mM CaCl","2 mM CaCl", "5 mM CaCl", "8 mM CaCl", "10 mM CaCl"],
            hue="Strain")
fig.savefig("All Strains CaCl and Antibiotic Conc with Expression Time.png", dpi=300)